In [2]:
%load_ext autoreload
%autoreload 2

In [15]:
from typing import Dict, List, Tuple

import numpy as np
from PIL import Image

from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from pysolotools.core.models import BoundingBox2DLabel, BoundingBox2DAnnotation
from pysolotools.core.models import BoundingBox3DLabel, BoundingBox3DAnnotation
from pysolotools.core.models import Frame, Capture

from scipy.spatial.transform import Rotation as R

from torchvision.datasets import ImageFolder
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
# from torch.utils.data import ConcatDataset, DataLoader

import torch
from torch import nn

import matplotlib.pyplot as plt

import utils
# %matplotlib ipympl

In [4]:
train_folder = 'D:/Unity/dataset/solo_22'
# training_dir = './data/train'
# testing_dir = './data/test'

In [256]:
solo = Solo(data_path='D:/Unity/dataset/solo_22')


def pos_to_pixel(capture, pos) -> np.ndarray:
    r = R.from_quat(capture.rotation)
    translation = np.array(capture.position)

    camera_coordinates = r.inv().apply((pos - translation).T).T #- translation
    pixel_coordinates = intrinsic @ camera_coordinates
    pixels = pixel_coordinates / pixel_coordinates[-1]
    return pixels[:2, 0].astype(np.int_)


for frame_idx, frame in enumerate(solo.frames()):
    # print(f'\r{frame_idx}/{len(solo.frames())}', end='')
    for i, capture in enumerate(frame.captures):
        resolution = np.array(capture.dimension)

        intrinsic = np.array([capture.matrix]).reshape((3, 3))
        intrinsic /= intrinsic[-1,-1]
        # intrinsic *= np.array([-resolution[0]/2, resolution[1]/2, 1])
        # print(intrinsic)
        # intrinsic[0, -1] = resolution[0]/2
        # intrinsic[1, -1] = resolution[1]/2

        translation = np.array(capture.position)
        # print(translation)

        gt_pixels = {}
        bboxes = [anno for anno in capture.annotations if isinstance(anno, BoundingBox2DAnnotation)][0]
        for bbox in bboxes.values:
            pixel = np.array(bbox.origin) + np.array(bbox.dimension)/2
            gt_pixels[bbox.instanceId] = pixel.astype(np.int_)

        r = R.from_quat(capture.rotation)
        pixels = {}
        anno_3d = [anno for anno in capture.annotations if isinstance(anno, BoundingBox3DAnnotation)][0]
        poses = []
        for u, anno in enumerate(anno_3d.values):
            t = np.array(anno.translation)
            pos = np.array(r.apply(t.T).T + translation)
            poses.append(pos)

            # if u == 1: break
            # print(pos)

        #     # camera_coordinates = r.inv().apply((pos - translation).T).T
        #     ueler = r.as_matrix()
        #     # print('pos', pos, pos - translation)
        #     camera_coordinates = (pos - translation) @ ueler
        #     # print(camera_coordinates)
        #     _intrinsic = intrinsic * np.array([-resolution[0]/2, resolution[1]/2, 1])
        #     # print(camera_coordinates.shape)
        #     pixel_coordinates = camera_coordinates @ _intrinsic
        #     # print(pixel_coordinates)
        #     # print(camera_coordinates.shape, intrinsic.shape)
        #     pixel = pixel_coordinates / pixel_coordinates[-1]
        #     pixel[:2] += resolution/2
        #     pixels[anno.instanceId] = pixel[:2].astype(np.int_)
        # poses = np.array(poses)
        # # print(poses.shape)
        # camera_coordinates = (poses - translation[np.newaxis, ...]) @ ueler
        # # print(camera_coordinates.shape)
        # pixel_coordinates = camera_coordinates @ _intrinsic
        # pixel = pixel_coordinates / pixel_coordinates[:, -1]
        # pixel[:, :2] += resolution/2
        # print(pixel[:, :2].astype(np.int_))
        
        camera = utils.Camera.from_unity(capture)
        print(
            camera(torch.tensor(poses), [1920, 1080, 1])
        )
        
        for key in gt_pixels.keys():
            print(gt_pixels[key]) #, pixels[key])
        # break
        print('-'*30)
    break



tensor([[1124,  617],
        [1136,  603],
        [ 816,  603]], dtype=torch.int32)
[1124  617]
[1137  603]
[815 604]
------------------------------
tensor([[759, 665]], dtype=torch.int32)
[755 667]
------------------------------


In [214]:
resolution

array([256., 256.])

In [ ]:
np.linalg.inv(intrinsic).dot((pixel - resolution)[:, 0])